In [1]:
import pickle as pkl
import json
import os
import numpy as np

In [2]:
import sys
sys.path.append("/home/ziniuw/zero-shot-cost-estimation")
from cross_db_benchmark.benchmark_tools.parse_run import parse_run
from cross_db_benchmark.benchmark_tools.database import DatabaseSystem

In [6]:
db = "imdb_full"
wl = "complex_workload_400k_s6_c8220.json"
source = "/flash1/ziniuw/zero-shot-data/runs/raw/" + db + "/" + wl
target = "/flash1/ziniuw/zero-shot-data/runs/parsed_plans/new_imdb_full/" + wl
parse_baseline = False
cap_queries = None
#no_plans, stats = parse_run(source, target, DatabaseSystem.POSTGRES, min_query_ms=100, cap_queries=cap_queries,
                             #   parse_baseline=parse_baseline, parse_join_conds=True)

In [7]:
parse_files = "/home/ziniuw/zero-shot-data/runs/parsed_plans/new_imdb_full/" + wl
with open(parse_files, "r") as f:
    all_info = json.load(f)
queries_sql = all_info['sql_queries']
print(len(queries_sql))

16904


In [3]:
def get_cardinality(q):
    ops = q['plan_parameters']['op_name']
    if ops in ['Finalize Aggregate', 'Gather', 'Aggregate', "Partial Aggregate"]:
        return get_cardinality(q["children"][0])
    else:
        true = q['plan_parameters']['act_card']
        est = q['plan_parameters']['est_card']
        #if true <= 3:
         #   print(f"{ops} with cardinality {true}")
        return true, est

In [8]:
queries = all_info['parsed_plans']

In [22]:
all_query_plans = [] 
all_queries_sql = []
runtimes = []

for workload in ["s4", "s5", "s6"]:
    file_name = f"complex_workload_400k_{workload}_c8220.json"

    parse_files = "/home/ziniuw/zero-shot-data/runs/parsed_plans/new_imdb_full/" + file_name
    with open(parse_files, "r") as f:
        all_info = json.load(f)
    queries = all_info['parsed_plans']
    query_sql = all_info['sql_queries']
    
    for i, q in enumerate(queries):
        runtimes.append(q["plan_runtime"])
        all_query_plans.append(q)
        true, est = get_cardinality(q)
        all_queries_sql.append([query_sql[i], true, est, q["plan_runtime"]])

print(len(all_query_plans))
print(len(runtimes))
print(len(all_queries_sql))

50722
50722
50722


In [23]:
runtimes = np.asarray(runtimes)
idx = np.argsort(runtimes)
testing_idx = []
for i in range(0, len(runtimes), 25):
    testing_idx.append(idx[i])
len(testing_idx)

2029

In [25]:
all_queries_testing = [all_queries_sql[i] for i in testing_idx]
all_query_plan_testing = [all_query_plans[i] for i in testing_idx]
print(len(all_queries_testing))
training_idx = [i for i in range(len(all_queries_sql)) if i not in testing_idx]
all_queries_training = [all_queries_sql[i] for i in training_idx]
all_query_plan_training = [all_query_plans[i] for i in training_idx]
print(len(all_queries_training))

2029
48693


In [26]:
file_name = "complex_workload_400k_s4_c8220.json"
parse_files = "/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/" + file_name
with open(parse_files, "r") as f:
    all_info = json.load(f)
queries = all_info['parsed_plans']

In [27]:
train_all_info = dict()
train_all_info['database_stats'] = all_info['database_stats']
train_all_info['run_kwargs'] = all_info['run_kwargs']
train_all_info['parsed_plans'] = all_query_plan_training

test_all_info = dict()
test_all_info['database_stats'] = all_info['database_stats']
test_all_info['run_kwargs'] = all_info['run_kwargs']
test_all_info['parsed_plans'] = all_query_plan_testing

In [ ]:
new_test_parsed_plans = []
result_path = "/flash1/ziniuw/CEB/zero-shot-results/MSCN3118603545/queries/zero-shot-test-preds/"
MSCN_est = dict()
for file in os.listdir(result_path):
    if file.endswith(".pkl"):
        query_no = int(file.split(".pkl")[0])
    new_test_parsed_plans.append(test_parsed_plans[query_no])
print(len(new_test_parsed_plans))

In [ ]:
test_all_info['parsed_plans'] = new_test_parsed_plans

In [28]:
with open('/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_queries_training_50k.json', 'w') as f:
    json.dump(train_all_info, f)

In [29]:
with open('/home/ziniuw/zero-shot-data/runs/parsed_plans/imdb_full/complex_queries_testing_2k.json', 'w') as f:
    json.dump(test_all_info, f)

In [30]:
with open('/home/ziniuw/zero-shot-data/runs/raw/imdb_full/complex_queries_training_50k.json', 'w') as f:
    json.dump(all_queries_training,f)

In [31]:
with open('/home/ziniuw/zero-shot-data/runs/raw/imdb_full/complex_queries_testing_2k.json', 'w') as f:
    json.dump(all_queries_testing,f)

In [ ]:
with open('complex_queries_testing_2k.json') as f:
    a = json.load(f)